In [5]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: e:\AIprojecttt\Medical-Generative-ChatBot\research


In [6]:
%pwd

'e:\\AIprojecttt\\Medical-Generative-ChatBot\\research'

In [7]:
import os
os.chdir("../")

In [8]:
%pwd

'e:\\AIprojecttt\\Medical-Generative-ChatBot'

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [11]:
extracted_data = load_pdf_file('Data/')

In [12]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 8706


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\Admin\AppData\Local\Temp\ipykernel_22708\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [85]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [111]:
API_KEY=os.environ.get('API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')


In [92]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=API_KEY)

index_name = "medicalbot"

# Check if the index exists
if index_name not in pc.list_indexes().names():
    # Create the index
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [110]:
import os
os.environ["API_KEY"] =API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print(API_KEY)

pcsk_7FpK1a_EzhACuBReqUoGLUxA3vza24vvPPBYkmYTGGbhnnhyiS8pTCVSTn3x7m6AWW4qbu


In [37]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(
    index_name="medicalbot",
    embedding=embeddings,
    pinecone_api_key=API_KEY
)

vectorstore.add_documents(text_chunks)

['682ab1a3-c97a-41bc-9f4d-112ec4ca8877',
 'd84ebe53-f18b-4a65-99b3-0742addf73bd',
 '4baa998f-88e6-4e40-accb-97d48cf7368e',
 '05f3064c-9b79-4ec3-8111-f79d03cf1114',
 '0cd2e6c4-dc3d-4e58-899b-7e1043de39fc',
 'f05f3fdd-8ac0-4fe6-9f6a-fe06f81ab6ae',
 '5b101c21-2c7a-4073-9882-4051acd3dc54',
 '4f397435-dc4d-47d2-91cb-ef88b8fa22d2',
 'cf3c5d43-91f2-4353-98a8-62b596008cc5',
 '9889e13e-e2db-4a28-890c-e1209f9eb5a5',
 '31ee356e-a04e-4596-a686-7865beb53a90',
 'bab88714-3448-4e32-8303-8b9b2537c6cf',
 '3bbdee7c-db19-4fe5-b66e-176974c764b5',
 'cda1f45f-ac79-48f3-84f0-d7bcefba0d37',
 'a7f18fa5-761a-4abe-8477-5f48832e8e7e',
 'e14b0431-ac49-46bd-a2dd-40d94dfbab42',
 '989f6c0a-bb9c-4804-8cc2-c49ae29c0994',
 'a3de2a7d-efc3-4407-a9a9-54479ae2c818',
 '0ba3770b-4283-48e7-8777-81f919b32d29',
 '9f3259ef-6c4f-4f9d-b141-43cb85bc2abd',
 'eadcf03c-de2f-4a6d-96a3-96ad2d4ee222',
 'aeb5e7e6-3e55-4057-ad5b-cb7ddd557ca1',
 '82e642ed-5c82-4dc0-953c-578f60342959',
 'd094ee0a-d71a-4c16-b405-682f8957c034',
 'a6b7953e-6049-

In [101]:
# Sample query
query = "What are the symptoms of diabetes?"

# Search in Pinecone index via Langchain wrapper
results = vectorstore.similarity_search(query, k=3)  # k = top 3 matches
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})
# Print the results
for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(result.page_content)


Result 1:
Diabetes is a complex disorder characterized by an
inability of the body to properly regulate the levels of
sugar and insulin (a hormone made by the pancreas) in the
blood. As diabetes progresses, the blood vessels that feed
the retina become damaged in different ways. The dam-
aged vessels can have bulges in their walls (aneurysms),
they can leak blood into the surrounding jelly-like material
(vitreous) that fills the inside of the eyeball, they can

Result 2:
cult to make that distinction. The diabetes-peripheral
neuropathy link has been well established. A typical pat-
tern of diabetes-associated neuropathic symptoms
includes sensory effects that first begin in the feet. The
associated pain or pins-and-needles, burning, crawling,
or prickling sensations form a typical “stocking” distrib-
ution in the feet and lower legs. Other diabetic neu-
ropathies affect the autonomic nerves and have potential-
ly fatal cardiovascular complications.

Result 3:
PERIODICALS
Zoorob, Roger

In [102]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define the assistant's behavior with clear instructions and a helpful tone
assistant_instructions = (
    "You are a helpful assistant designed to answer questions based on provided context. "
    "Use the retrieved information below to construct your response. "
    "If the answer isn't present in the context, it's okay to say you don't know. "
    "Keep your reply concise and limited to three sentences."
    "\n\n"
    "{context}"
)

# Build the prompt template using a system message and user input
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", assistant_instructions),
        ("human", "{input}"),
    ]
)


In [103]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [108]:
response = rag_chain.invoke({"input": "what is acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [109]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}